## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [15]:
# 1. Import the WeatherPy_database.csv file. 
file_to_open = os
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,60.39,91,99,3.06,moderate rain
1,1,Cape Town,ZA,-33.9258,18.4232,88.21,26,0,13.80,clear sky
2,2,Dwarka,IN,22.2394,68.9678,86.70,76,15,16.53,few clouds
3,3,Nuuk,GL,64.1835,-51.7216,45.21,57,0,3.44,clear sky
4,4,Rio Gallegos,AR,-51.6226,-69.2181,37.45,60,0,11.50,clear sky


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,ZA,-33.9258,18.4232,88.21,26,0,13.80,clear sky
2,2,Dwarka,IN,22.2394,68.9678,86.70,76,15,16.53,few clouds
8,8,Lingao,CN,19.9103,109.6859,80.87,86,100,3.74,overcast clouds
11,11,Hermanus,ZA,-34.4187,19.2345,87.80,36,0,3.80,clear sky
16,16,Lata,PT,40.1629,-8.3327,81.48,43,37,10.78,scattered clouds
17,17,Cockburn Town,TC,21.4612,-71.1419,80.35,86,100,17.69,overcast clouds
30,30,Gilgit,PK,35.9221,74.3087,83.30,27,37,0.89,scattered clouds
36,36,Georgetown,MY,5.4112,100.3354,83.95,77,20,6.91,few clouds
43,43,Mahajanga,MG,-15.7167,46.3167,91.42,35,20,11.50,few clouds
53,53,Hithadhoo,MV,-0.6000,73.0833,83.55,66,91,2.59,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                157
City                   157
Country                155
Lat                    157
Lng                    157
Max Temp               157
Humidity               157
Cloudiness             157
Wind Speed             157
Current Description    157
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,ZA,-33.9258,18.4232,88.21,26,0,13.80,clear sky
2,2,Dwarka,IN,22.2394,68.9678,86.70,76,15,16.53,few clouds
8,8,Lingao,CN,19.9103,109.6859,80.87,86,100,3.74,overcast clouds
11,11,Hermanus,ZA,-34.4187,19.2345,87.80,36,0,3.80,clear sky
16,16,Lata,PT,40.1629,-8.3327,81.48,43,37,10.78,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
685,685,Bhum,IN,18.4667,75.6667,98.53,23,36,9.91,scattered clouds
690,690,Acarau,BR,-2.8856,-40.1200,81.09,72,66,11.14,broken clouds
692,692,Buala,SB,-8.1450,159.5921,80.47,79,98,9.48,light rain
693,693,Kaniama,CD,-7.5667,24.1833,85.73,25,100,5.10,overcast clouds


In [36]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,88.21,clear sky,-33.9258,18.4232,
2,Dwarka,IN,86.70,few clouds,22.2394,68.9678,
8,Lingao,CN,80.87,overcast clouds,19.9103,109.6859,
11,Hermanus,ZA,87.80,clear sky,-34.4187,19.2345,
16,Lata,PT,81.48,scattered clouds,40.1629,-8.3327,
17,Cockburn Town,TC,80.35,overcast clouds,21.4612,-71.1419,
30,Gilgit,PK,83.30,scattered clouds,35.9221,74.3087,
36,Georgetown,MY,83.95,few clouds,5.4112,100.3354,
43,Mahajanga,MG,91.42,few clouds,-15.7167,46.3167,
53,Hithadhoo,MV,83.55,overcast clouds,-0.6000,73.0833,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,88.21,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
2,Dwarka,IN,86.70,few clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
8,Lingao,CN,80.87,overcast clouds,19.9103,109.6859,Lingao Ditai Inn
11,Hermanus,ZA,87.80,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
16,Lata,PT,81.48,scattered clouds,40.1629,-8.3327,Casa Maquia
...,...,...,...,...,...,...,...
685,Bhum,IN,98.53,scattered clouds,18.4667,75.6667,धनंजय होम
690,Acarau,BR,81.09,broken clouds,-2.8856,-40.1200,castelo pizzaria
692,Buala,SB,80.47,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
693,Kaniama,CD,85.73,overcast clouds,-7.5667,24.1833,


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ['Hotel Name'])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,88.21,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
2,Dwarka,IN,86.70,few clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
8,Lingao,CN,80.87,overcast clouds,19.9103,109.6859,Lingao Ditai Inn
11,Hermanus,ZA,87.80,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
16,Lata,PT,81.48,scattered clouds,40.1629,-8.3327,Casa Maquia
...,...,...,...,...,...,...,...
685,Bhum,IN,98.53,scattered clouds,18.4667,75.6667,धनंजय होम
690,Acarau,BR,81.09,broken clouds,-2.8856,-40.1200,castelo pizzaria
692,Buala,SB,80.47,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
693,Kaniama,CD,85.73,overcast clouds,-7.5667,24.1833,


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))